# Setup

In [49]:
raise RuntimeError("Don't run this anymore!!!! We have enough users for this experiment...")

RuntimeError: Don't run this anymore!!!! We have enough users for this experiment...

In [2]:
%%vertica
SELECT 'Connected to Vertica!' AS ''

,
0,Connected to Vertica!


In [3]:
%%vertica
SET TIME ZONE to UTC

In [4]:
%%query [lms]
SELECT 'Connected to the LMS read replica!' AS ''

,
0,Connected to the LMS read replica!


In [5]:
import datetime

# Configuration

## Enrollment Date Config

In [6]:
from dateutil.tz import tzutc

# enter enrollment start date
enrollment_start = datetime.datetime(2017, 7, 19, 18, 40, tzinfo=tzutc())

# Make sure we start our window on a Wednesday
assert(enrollment_start.isoweekday() == 3)

## Course Config

In [7]:
# enter course IDs, if only a subset of courses are included in the experiment
# e.g., course_ids = ('course-v1:ANUx+ANU-ASTRO1x+2T2016',)
# course_ids = ()
course_ids = (
    'course-v1:IIMBx+AC103x+2T2017',
    'course-v1:AdelaideX+Project101x+1T2017',
    'course-v1:DelftX+LfE101x+2T2017',
    'course-v1:ColumbiaX+DS101X+1T2017',
    'course-v1:PennX+SD2x+2T2017',
    'course-v1:PennX+SD1x+2T2017',
    'course-v1:HarvardX+PH525.1x+2T2017',
    'course-v1:MichiganX+UX501x+3T2016',
    'course-v1:UBCx+Marketing1x+3T2015',
)
len_courses = len(course_ids)

## Segmentation Config

In [8]:
# enter number of user segments (i.e., total number of treatment groups + 1 for the control group)
num_user_segments = 2

## LMS Config

# Setup post-config

In [55]:
# compute enrollment start date
enrollment_end = enrollment_start + datetime.timedelta(days=14)
print(
    '\033[1m' + str(enrollment_start) + '\033[0m ' +
    '<= Enrollment Date <= ' +
    '\033[1m' + str(enrollment_end) + '\033[0m'
)

# create dummy course-tuple to satisfy SQL query syntax
course_filter = course_ids if len_courses else ('dummy',)

2017-07-19 18:40:00+00:00 <= Enrollment Date <= 2017-08-02 18:40:00+00:00


# Segmented Enrollments

## Load segmentation data from the LMS

In [10]:
import json
import ciso8601
import pandas as pd

already_segmented = %query [lms] SELECT exp.key, exp.value, exp.user_id FROM experiments_experimentdata AS exp WHERE experiment_id = 4

already_segmented['course_id'] = already_segmented['key'].apply(lambda key: key.replace('cohort_availability_date.', ''))
already_segmented['segment'] = already_segmented['value'].apply(lambda value: json.loads(value)['segment'])
already_segmented['cohort_availability'] = already_segmented['value'].apply(lambda value: ciso8601.parse_datetime(json.loads(value)['cohort_availability']).date())
already_segmented['segmentation_reason'] = already_segmented['value'].apply(lambda value: json.loads(value).get('segmentation_reason', 'visit'))
already_segmented['user_id'] = already_segmented['user_id'].astype(int)
del already_segmented['key']
del already_segmented['value']

In [11]:
len(already_segmented)

8414

## Load any newly content-available users from the enrollments table

In [53]:
%%query [lms] (enrollment_start, enrollment_end, course_ids) -> enrollments
SELECT
    sce.user_id,
    sce.course_id,
    sce.created AS content_availability,
    au.username
FROM
    student_courseenrollment sce
JOIN auth_user au ON au.id = sce.user_id
JOIN user_api_userpreference up ON up.`key`='pref-lang' AND up.value='en' AND up.user_id = sce.user_id
LEFT JOIN experiments_experimentdata AS exp ON exp.user_id = sce.user_id AND exp.experiment_id = 4
WHERE
        sce.created > :enrollment_start
    AND sce.created < CAST(:enrollment_end AS DATE)
    AND sce.course_id IN :course_ids
    AND exp.user_id IS NULL

In [28]:
len(enrollments)

2282

In [29]:
import pandas as pd
enrollments['content_availability'] = pd.to_datetime(enrollments.content_availability)

In [30]:
print('date, previous_wednesday')
for test_date_offset in range(16):
    dt = datetime.date(2017,7,18) + datetime.timedelta(days=test_date_offset)
    offset = (dt.isoweekday() + 4) % 7
    print(dt, dt - datetime.timedelta(days=offset))

date, previous_wednesday
2017-07-18 2017-07-12
2017-07-19 2017-07-19
2017-07-20 2017-07-19
2017-07-21 2017-07-19
2017-07-22 2017-07-19
2017-07-23 2017-07-19
2017-07-24 2017-07-19
2017-07-25 2017-07-19
2017-07-26 2017-07-26
2017-07-27 2017-07-26
2017-07-28 2017-07-26
2017-07-29 2017-07-26
2017-07-30 2017-07-26
2017-07-31 2017-07-26
2017-08-01 2017-07-26
2017-08-02 2017-08-02


In [31]:
enrollments['cohort_availability'] = enrollments.content_availability.apply(lambda ts: ts.date() - datetime.timedelta((ts.date().isoweekday() + 4) % 7))

In [32]:
unsegmented_users = enrollments[['user_id','course_id','cohort_availability','username']].copy()

In [33]:
unsegmented_users['segmentation_reason'] = 'enrollment'

In [34]:
import numpy as np
unsegmented_users['segment'] = np.random.randint(num_user_segments, size=len(unsegmented_users))

In [35]:
unsegmented_users.groupby('course_id').user_id.count()

course_id
course-v1:AdelaideX+Project101x+1T2017    242
course-v1:ColumbiaX+DS101X+1T2017         153
course-v1:DelftX+LfE101x+2T2017           578
course-v1:HarvardX+PH525.1x+2T2017        373
course-v1:IIMBx+AC103x+2T2017             125
course-v1:MichiganX+UX501x+3T2016          74
course-v1:PennX+SD1x+2T2017               148
course-v1:PennX+SD2x+2T2017                94
course-v1:UBCx+Marketing1x+3T2015         495
Name: user_id, dtype: int64

In [36]:
unsegmented_users.groupby(['course_id', 'segment']).user_id.count()

course_id                               segment
course-v1:AdelaideX+Project101x+1T2017  0          117
                                        1          125
course-v1:ColumbiaX+DS101X+1T2017       0           68
                                        1           85
course-v1:DelftX+LfE101x+2T2017         0          271
                                        1          307
course-v1:HarvardX+PH525.1x+2T2017      0          183
                                        1          190
course-v1:IIMBx+AC103x+2T2017           0           64
                                        1           61
course-v1:MichiganX+UX501x+3T2016       0           45
                                        1           29
course-v1:PennX+SD1x+2T2017             0           80
                                        1           68
course-v1:PennX+SD2x+2T2017             0           42
                                        1           52
course-v1:UBCx+Marketing1x+3T2015       0          277
                 

In [37]:
unsegmented_users.groupby(['segment']).user_id.count()

segment
0    1147
1    1135
Name: user_id, dtype: int64

In [38]:
unsegmented_users.groupby(['cohort_availability']).user_id.count()

cohort_availability
2017-07-19       1
2017-07-26    2281
Name: user_id, dtype: int64

# LMS

In [39]:
import edx_secret

environment = 'stage'

if environment == 'stage':
    server_base_url = 'https://courses.stage.edx.org'
elif environment == 'prod':
    server_base_url = 'https://courses.edx.org'
else:
    raise(f'Unrecognized environment: {environment}')

oauth2_client_id = edx_secret.read_secret_from_env_or_prompt(f'OPS_{environment.upper()}_EDX_OAUTH_CLIENT_ID')
oauth2_client_secret = edx_secret.read_secret_from_env_or_prompt(f'OPS_{environment.upper()}_EDX_OAUTH_CLIENT_SECRET')

In [40]:
from edx_rest_api_client.client import EdxRestApiClient

EXPERIMENT_ID = 4

access_token_url = server_base_url.strip('/') + '/oauth2/access_token/'
access_token, expires = EdxRestApiClient.get_oauth_access_token(
    access_token_url, oauth2_client_id, oauth2_client_secret, token_type='jwt')
print('JWT access token expires at: {}'.format(expires))

experiments_api_url = server_base_url.strip('/') + '/api/experiments/v0/'
client = EdxRestApiClient(experiments_api_url, jwt=access_token)

print(f'Connected to {server_base_url}')

JWT access token expires at: 2017-08-02 23:13:32.451260
Connected to https://courses.edx.org


## Insert Experiment Data

In [96]:
import csv
import io

csv_text = """course_key,week,bullet1,bullet2,bullet3
course-v1:AdelaideX+Project101x+1T2017,1,You will learn what project management is,You will be introduced to a number of different project management scenarios which we will refer to throughout the course,Did you know that there are 5 characterisitics of good project managers? Find out what these are determine your own areas for development 
course-v1:AdelaideX+Project101x+1T2017,2,Learn about one of the most important phases of project management - the initiation phase.,You will be introduced to the project management canvas that will help you work out what problem your project needs to solve,Learn how to find out what the project requires you to do and help define the start and end dates of your project
course-v1:AdelaideX+Project101x+1T2017,3,Understand the planning phase of your project,"Learn more about the triple constraints to any project; quality, cost and time",Learn more about estimating the cost of your project and planning timeframes
course-v1:AdelaideX+Project101x+1T2017,4,Learn how to determine the risks of your project and how to respond to these,"Learn about the different kinds of risk that include technical, financial, commercial or reputational risk",Do you know what the acronym PESTLE stands for and how it relates to project management risk. Join us next week to find out.
course-v1:AdelaideX+Project101x+1T2017,5,Learn about project teams and communication,Learn how to engage and manage your project stakeholders and how to resource your project team,Find out how to use your communication tools effectively
course-v1:AdelaideX+Project101x+1T2017,6,It's time for project closure and handover,Learn the importance of measuring success and how to evaluate your project,Fin out about the key steps you need to take as part of your project closure
course-v1:ColumbiaX+DS101X+1T2017,1,"A team of experts will introduce the field of Data Science, and discuss its role in driving innovation across a multitude of domains.","We will also discuss the importance of data visualization in data science approaches, and the type of skills data-scientists, as well as non data-scientists, need in a world that is literally exploding with data. ",An opportunity to discuss a statistical question about this online course on the forums.
course-v1:ColumbiaX+DS101X+1T2017,2,"We will dive right into the statistical foundation that underpins data science and analytics with discussions on data collection, analysis and inference processes.",A variety of interactive graded and ungraded exercises will help reinforce this week's topics.,Discuss the questions posed this week by our team of experts on the forums.
course-v1:ColumbiaX+DS101X+1T2017,3,"We will continue the discussion on statistics and probability by reviewing the topics of conditional probability, studying association and regression analysis. ","Our team will give of some of the themes at the forefront of Data Science, including topic modeling, clustering text and metrics for label description.","This week's assignments include a variety of graded and ungraded exercises, as well as the questions on the forums."
course-v1:ColumbiaX+DS101X+1T2017,4,"We will cover the principles and practices used in exploratory data analysis and visualization, drawing upon several real-world examples from the social sciences, health-care, as well as sports.",Interactive graded and ungraded exercises on exploratory data analysis and visualization.,An opportunity to discuss this week's questions on the forums.
course-v1:ColumbiaX+DS101X+1T2017,5,"In this module, we will introduce the philosophy of Bayesian inference and Bayesian modeling techniques using illustrative case studies.",Several examples will illustrate Bayesian modeling in action.,"A variety of exercises on Bayesian modeling, as well as an opportunity to discuss this topic on the forums."
course-v1:PennX+SD1x+2T2017,1,Getting started with the basics of Java.,"You'll learn about variables, conditions and loops.",You'll create your first class in Java.
course-v1:PennX+SD1x+2T2017,2,Testing code while you write it in order to ensure that fewer bugs make it all the way to a client.,Become an effective debugger by using tools that are within Eclipse.,"Learn how to handle things like bad inputs/wrong file formats, etc."
course-v1:PennX+SD1x+2T2017,3,You'll start getting into intermediate Java.,"You'll learn about arrayLists, encapsulation, and static methods.",You'll learn how objects interact with other objects.
course-v1:PennX+SD1x+2T2017,4,"Polymorphism (one name, many forms), which is generally considered one of the core principles of object oriented programming.","Using overloaded methods, class inheritance, and interfaces to ensure the least amount of code gets repeated.",Programming your own Battleship game!
course-v1:PennX+SD2x+2T2017,1,"An introduction and overview of core data structures like linked lists, hash sets, and maps",A comparison of strengths and weaknesses of each data structure reviewed,Applications of use for data structures through HTML validation and sentiment analysis of a body of text
course-v1:PennX+SD2x+2T2017,2,An introduction and overview of advanced data structures like binary search trees and heaps,A discussion of graph data structure and common applications,You will have a chance to practice with graph traversal algorithms and programmatic data input
course-v1:PennX+SD2x+2T2017,3,You will learn a system of documenting software design,Explore and understand the need for good design,Practice using design principles to achieve high-quality software
course-v1:PennX+SD2x+2T2017,4,Explore ways to make existing code more reliable,A discussion of best practices to increase software understandability,You'll see how the application of efficiency principles can decrease runtime
course-v1:UBCx+Marketing1x+3T2015,1,You will learn about examples that illustrate how marketing has continued to change and evolve over time,You will learn how competition impacts the dynamic landscape of marketing,"You will be able to adopt useful marketing tools and frameworks (the 3Cs, SWOT framework, and PEST analysis) into your own practice"
course-v1:UBCx+Marketing1x+3T2015,2,You will learn about strategies and techniques for getting to know your consumer,You learn how to base marketing decisions on the insights you gain from market research,You will learn how to target different segments of the market by using frameworks and tools that identify the consumers most likely to respond 
course-v1:UBCx+Marketing1x+3T2015,3,"You will learn about positioning: understanding who your consumer is, and how to identify points of parity and points of difference between your own brand and that of the competition.","You will be able to construct a brand positioning statement that emphasizes the value proposition that you are offering to the consumer, and will underpin the approach and communication strategies that you employ in your marketing efforts.","You will learn about frameworks and considerations around how to define your product, and will also gain an understanding of the product lifecycle from a marketing perspective in order to develop marketing strategies that can minimize market phases of the product that are costly, and extend phases that are profitable."
course-v1:UBCx+Marketing1x+3T2015,4,You will learn about the value of brands and how they communicate meaning and identity to consumers,"You will learn how brands are constructed not only by the companies that build them, but also by the conversations, interactions, and experiences that consumers have with brands.","You will learn about pricing: how companies make decisions around setting prices, what criteria and considerations go into decisions made around pricing, and how understanding the existing market can inform your pricing strategy."
course-v1:UBCx+Marketing1x+3T2015,5,"You will be looking at distribution from a marketing perspective, with a particular focus on how the Internet has disrupted traditional notions of distribution, and has changed the way we think about how products get to consumers. ","You will learn about a few different distribution strategies, and factors to consider when making decisions about distribution.","You will learn about Integrated Marketing Communications (IMC), different communication options that are available to you, how to go about understanding how your brand is perceived, and what approaches you should use in communicating with your consumer."
course-v1:UBCx+Marketing1x+3T2015,6,"You will learn how to monitor and facilitate the online conversations that are going on around your brand, and how to identify and leverage marketing opportunities that social media offer","You will also be introduced to some of the data, metrics, and measures that are used in marketing. Such measurements are used to evaluate efficacy in marketing efforts, and to inform future marketing decisions and strategies.","We will look ahead to the future of marketing, and discuss a few cases of things that are happening in the marketplace today that may give us a glimpse of what is to come."
course-v1:HarvardX+PH525.1x+2T2017,1,"This week will be lighter than the following weeks, but it’s still very important!","We want to get everyone set up using R and RStudio, so that you are ready and organized for data science.","This will be easy if you’re already familiar with the R language, but will otherwise help get you up to speed for the following weeks."
course-v1:HarvardX+PH525.1x+2T2017,2,the concept of a random variable,what it means to talk about the null distribution,"probability distributions in general, and the widely used normal distribution"
course-v1:HarvardX+PH525.1x+2T2017,3,"This week we have formally arrived at one of the most important topics in statistics, and in the life sciences: statistical inference.",Inference is the use of probability to learn features of a population from data.,A typical example is deciphering if two groups (for example cases versus controls) are different on average.
course-v1:HarvardX+PH525.1x+2T2017,4,"In our last week, we will cover two very important topics that will help us determine if the methods described in the previous weeks are appropriate and provide alternatives for when they are not: Exploratory Data Analysis (EDA) and robust statistics.","EDA refers to using visualization to examine patterns in the data, and particularly if our assumptions about the data, which are necessary for inference, hold true.","In the second part, we will discuss a number of robust statistical techniques, which are useful in particular when the assumptions used in “classical” statistical inference do not hold."
course-v1:MichiganX+UX501x+3T2016,1,Fundamental insights into the most basic question of this course: what is User Experience?,"Different aspects of the UX Research and Design process, including iterative prototyping, utilizing a user-centered approach, and understanding basic human behavior.",You will identify good and bad user interface designs and be able to articulate why.
course-v1:MichiganX+UX501x+3T2016,2,"An introduction to the three key techniques of UX Research: asking, observing, and inspecting.",The benefits of user tests and how to conduct them.,You will plan and develop scripting for your own Micro-Usability Test.
course-v1:MichiganX+UX501x+3T2016,3,Insights into how the UX Research process is applied in real world projects for businesses.,How information learned from UX Research can be applied to optimize user experience.,You will conduct and report on your own Micro-Usability Test
course-v1:MichiganX+UX501x+3T2016,4,"Learn the basic principles of the UX Design Process and how they align within the Assess, Design, and Build framework learned in Week 1.",The central role that Sketching and Prototyping play in UX Design.,You will begin sketching your own designs to generate solutions to a provided problem.
course-v1:MichiganX+UX501x+3T2016,5,How UX Design operates in the business world through an interview with a current UX Designer.,An (incomplete) history of UX and its rise with the era of personal computing.,You will practice the UX Design process by iterating on earlier sketch solutions. 
course-v1:IIMBx+AC103x+2T2017,1,"Introduction to basic accounting terms, how to record a transaction","Introduction to financial statements like balance sheet, profit and loss account, cash flow statement.","Regulations in accounting, users of these financial statements- who will use these and for what?"
course-v1:IIMBx+AC103x+2T2017,2,How to read and understand balance sheet - assets and liabilities,How to read and understand PL account - income and expenses,"How to read and understand cash flow statement - cash flow from operations, invenstments, financing"
course-v1:IIMBx+AC103x+2T2017,3,"Knowing the financial health of a company by calculating ratios like ROI, ROTA, etc. ","Asset management, cost management, leverage, tax and risk management.","Additional resources: Readymade excel template (Du Pont Chart) for ratio analysis, excel with data of 500 companies listed in NSE,India"
course-v1:IIMBx+AC103x+2T2017,4,Learn how firms organize cost data to derive product or service cost,"Types of costing: Job costing, Process costing and Customer costing.
How to allocate indirect costs.","
Activity Based Costing - a new method that completely changes the decision making process"
course-v1:IIMBx+AC103x+2T2017,5,"Absorption costing vs Marginal costing, how it will influence decision making",How to compute break-even point for multi product firm,Optimal product mix and optimal resource allocation are the key things to maximize profit
course-v1:IIMBx+AC103x+2T2017,6,"How accounting information is used to plan and control the expenses- Budgeting, types of budget and how to prepare a budget (exercise)",Variance analysis explains how to collect actual cost and compare with budgeted cost at macro and micro level.,"Types of viariance - Fixed, variable, overhead, material mix, slaes mix, yield variance. "
course-v1:DelftX+LfE101x+2T2017 ,1,The context of leaders,Multi-issue games,Assignment: suggest strategies for the mayor
course-v1:DelftX+LfE101x+2T2017 ,2,Welcome to the leadership mindset,Task: convince others,Finding your strengths and personal charter
course-v1:DelftX+LfE101x+2T2017,3,Exploring personal leadership characteristics,The positive impact of gratitude,Reflection on your own performance
course-v1:DelftX+LfE101x+2T2017,4,Bringing context and you together,Leadership models,Reflect on leadership styles
course-v1:DelftX+LfE101x+2T2017,5,Working on a framework for your career,Why asking for support is essential,Defining your search strategy
"""

# NOTE For demo purposes only. This won't affect production, but isn't necessary.
csv_text += 'edX/DemoX/Demo_Course,0,Learn about edX,Do some stuff,Profit?'

csv_data = io.StringIO(csv_text)



# NOTE (CCB): We use getattr here due to the hyphen in the URL. Oops.
key_value_client = getattr(client, 'key-value')

# TODO Pull data from CSV/Google Doc
weekly_content_template = """<ul class="ret-pacing-weekly-content" data-week-number="{week_number}">
  <li>{bullet_1}</li>
  <li>{bullet_2}</li>
  <li>{bullet_3}</li>
</ul>"""
data = []

reader = csv.DictReader(csv_data)
for row in reader:
    course_id = row['course_key'].strip()
    week_number = int(row['week'].strip()) - 1
    
    data.append({
        'experiment_id': EXPERIMENT_ID,
        'key': 'content.{course_id}.{week_number}'.format(course_id=course_id, week_number=week_number),
        'value': weekly_content_template.format(
            week_number=week_number,
            bullet_1=row['bullet1'].strip(),
            bullet_2=row['bullet2'].strip(),
            bullet_3=row['bullet3'].strip(),
        ),
    })


# TODO Uncomment this to update data
#key_value_client.bulk_upsert.put(data)

## Insert User Data

In [42]:
len(unsegmented_users)

2282

In [46]:
import json
from pprint import pprint as pp
from slumber.exceptions import HttpClientError
import ipywidgets as widgets
from IPython.display import display

data = [{
        'experiment_id': EXPERIMENT_ID,
        'user': row.username,
        'key': 'cohort_availability_date.' + row.course_id,
        'value': json.dumps({
            'cohort_availability': datetime.datetime.strftime(row.cohort_availability, '%Y-%m-%dT%H:%M:%S.000Z'),
            'segment': int(row.segment),
            'segmentation_reason': row.segmentation_reason
        })
    } for row in unsegmented_users.itertuples()]

progress = widgets.IntProgress(min=0, max=len(data))
display(progress)

succeeded = widgets.IntText(value=0, description='Succeeded: ')
display(succeeded)
failed = widgets.IntText(value=0, description='Failed: ')
display(failed)

from collections import defaultdict
errors = defaultdict(int)

for learner in data:
    try:
        client.data.post(learner)
        succeeded.value += 1
    except HttpClientError as ex:
        failed.value += 1
        error = ex.response.json()
        for ek, ev in error.items():
            errors[ek] += len(ev)
    finally:
        progress.value += 1

pp(dict(errors))

{'non_field_errors': 10}


In [136]:
import pytz

# Update data for demo course on stage.
usernames = ('nasthagiriEdX', 'clintonb','gabeatedx')
start_date = datetime.datetime(year=2017, month=7, day=19, tzinfo=pytz.UTC)

data = [{
    'experiment_id': EXPERIMENT_ID,
    'user': username,
    'key': 'cohort_availability_date.edX/DemoX/Demo_Course',
    'value': json.dumps({
        'cohort_availability': datetime.datetime.strftime(start_date, '%Y-%m-%dT%H:%M:%S.000Z'),
        'segment': 1,
        'segmentation_reason': 'enrollment'
    })
} for username in usernames]

from slumber.exceptions import HttpClientError
for learner in data:
    try:
        client.data.post(learner)
    except HttpClientError as ex:
        print(ex.response.status_code, ex.content.decode('utf8'))

400 {"non_field_errors":["The fields user, experiment_id, key must make a unique set."]}
400 {"non_field_errors":["The fields user, experiment_id, key must make a unique set."]}
400 {"non_field_errors":["The fields user, experiment_id, key must make a unique set."]}
